In [2]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import urllib.parse
import psycopg2

In [3]:
conn = psycopg2.connect(database = "DataLab", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "",
                        port = 5432)

cur = conn.cursor()
cur.execute("""SELECT ElementValue FROM programvariables WHERE ElementName = 'check_in_form_id'""")
tokenList = cur.fetchall()
conn.commit()
cur.close()
conn.close()
googleFormID = tokenList[0]

In [4]:
lastDateRun = datetime.now() - timedelta(days = 90) #this variable will be the last time we ran the job
query = f"SELECT * WHERE A > date '{lastDateRun.strftime('%Y-%m-%d')}'"
queryEncoded = urllib.parse.quote(query)

In [5]:
url = f'https://docs.google.com/spreadsheets/d/1feO9_9mFJyG6__FpjJTloaN1iIErCht29ykQ8Yuddy8/gviz/tq?tqx=out:csv&headers=1&tq={queryEncoded}'
response = requests.get(url)
content = response.content.decode()
checkIn = pd.read_csv(StringIO(content))
checkIn

,Timestamp,Email Address,Your First name,Your Last Name,Your UArizona Department (or Institution if applicable)
0,1/16/2024 13:50:06,crosales@cmm.uchile.cl,Cesar Alberto,Rosales-Alcantar,No Applicable
1,1/16/2024 13:55:44,francisca.gutierrez@ug.uchile.cl,Francisca,Gutierrez,No applicable
2,1/16/2024 13:55:47,hwaltz@arizona.edu,Helen,Waltz,College of Agriculture and Life Sciences
3,1/16/2024 13:56:28,nferrante@arizona.edu,Nick,Ferrante,Data Science/School of Information
4,1/16/2024 13:59:29,vmilici@gmail.com,Valerie,Milici,Ecology and evolutionary biology
...,...,...,...,...,...
247,2/26/2024 14:05:47,cgbilinsk@gmail.com,Christopher,Bilinski,Steward Observatory
248,2/26/2024 14:50:27,ayeshakhatun@arizona.edu,Ayesha,Khatun,Information Science
249,2/27/2024 13:59:50,caparicio@arizona.edu,Charlene,Aparicio`,ACBS
250,2/27/2024 14:39:59,sdmitrovic@arizona.edu,Sanja,Dmitrovic,Optical Sciences


In [56]:
def getWorkshopID(time, conn, cur):
    #Fetch the series ID for the day of the week because this will determine the time frame we check in
    cur.execute("""SELECT seriesID FROM series WHERE EXTRACT(DOW FROM startdate::date) = EXTRACT(DOW FROM %s::date)
                """, (time,))
    match = cur.fetchall()
    conn.commit()

    #Check series to see if this day of a workshop is a day with 2 workshops back to back or not as that changes our time frame we can search in
    if (len(match) == 2):
        cur.execute("""SELECT startTime, endTime FROM series 
                WHERE seriesID = %s OR seriesID = %s 
                """, (match[0][0], match[1][0]))
        timeRange = cur.fetchall()
        conn.commit()
        startTime1 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[0][0] )
        endTime1 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[0][1])  
        startTime2 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[1][0])
        endTime2 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[1][1])

        #Workshop 1 ends right before Workshop 2 begins so the ranges cannot overlap
        if ((endTime1 - startTime2) < timedelta(minutes=1)):
            cur.execute("""SELECT seriesID FROM series 
                WHERE (seriesID = %s AND ((StartTime - interval '15 minutes') < time %s) AND (EndTime > time %s))
                OR (seriesID = %s AND (StartTime < time %s) AND ((EndTime + interval '15 minutes') > time %s))
                """, (match[0][0], time, time, match[1][0], time, time))
            
        #Workshop 2 ends right before Workshop 1 begins so the ranges cannot overlap
        elif((endTime2 - startTime1) < timedelta(minutes=1)):
            cur.execute("""SELECT seriesID FROM series 
                WHERE (seriesID = %s AND ((StartTime - interval '15 minutes') < time %s) AND (EndTime > time %s))
                OR (seriesID = %s AND (StartTime < time %s) AND ((EndTime + interval '15 minutes') > time %s))
                """, (match[1][0], time, time, match[0][0], time, time))
        
        #The workshops are just on the same day and do not collide
        else:
            cur.execute("""SELECT seriesID FROM series 
                WHERE ( (seriesID = %s) OR (seriesID = %s) ) AND ( ((StartTime - interval '15 minutes') < time %s) AND ((EndTime + interval '15 minutes') > time %s)) )
                """, (match[0][0], match[1][0], time, time))

    else:
        cur.execute("""SELECT SeriesID FROM series WHERE ((StartTime - interval '15 minutes') < time %s) AND ((EndTime + interval '15 minutes') > time %s)
                    """, (time, time))
    seriesID = cur.fetchall()
    conn.commit()

    if (seriesID):
        cur.execute("""SELECT WorkshopID FROM workshops 
                    WHERE (WorkshopDate = date %s) AND SeriesID = %s
                    """, (time, seriesID[0][0]))
        workshopID = cur.fetchall()
        conn.commit()

        return workshopID[0][0]
    else:
        return -1

In [24]:
def uploadCheckIn(row, workshopID, conn, cur):

    #Fetch the supposed ID from the has function
    cur.callproc('hashRegistree', (row.iloc[2],))
    hashedNum = cur.fetchone()[0]
    conn.commit()

    #See if the RegID is in the databse already
    cur.execute("""SELECT RegID FROM registreeInfo WHERE RegID = %s""", (hashedNum,))
    match = cur.fetchall()
    conn.commit()

    #If the list isnt empty then a match was found so we will return the RegID
    if (len(match) != 0):
        cur.execute("""
                    UPDATE RegistreeWorkshops
                    SET CheckedIn = TRUE
                    WHERE RegID = %s AND WorkshopId = %s
                    """, (hashedNum, workshopID))
        conn.commit()
    #If the list is empty, there is no registrant matching the workshop so we will need to create an entry for them
    else:
        #Create a registreeInfo entry for the person
        cur.execute("""
                    INSERT INTO registreeInfo (RegID, FirstName, LastName, NetID, Email, College, Department, Major)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT DO NOTHING;
                    """, (hashedNum, row.iloc[0], row.iloc[1], None, row.iloc[2], row.iloc[3], None, None))
        conn.commit()

        #Create an entry for the person and the specific workshop they attended
        cur.execute("""
                    INSERT INTO RegistreeWorkshops (RegID, WorkshopID, Registered, CheckedIn)
                    VALUES (%s, %s, %s, %s)
                    """, (hashedNum, workshopID, False, True))
        conn.commit()   

In [ ]:
250677359474826468

In [58]:
test = checkIn.copy()
test.columns = ['checkTime', 'Email', 'FirstName', 'LastName', 'Department']
test = test[['FirstName', 'LastName', 'Email', 'Department', 'checkTime']]

conn = psycopg2.connect(database = "DataLab", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "",
                        port = 5432)
cur = conn.cursor()

for _,row in test.iterrows():
    workshopID = getWorkshopID(row.iloc[4], conn, cur)
    if (workshopID > 0):
        uploadCheckIn(row, workshopID, conn, cur)

cur.close()
conn.close()    

In [44]:
series1 = checkIn.loc[32]['Timestamp']
series2 = checkIn.loc[72]['Timestamp']

In [45]:
series2

'1/24/2024 12:47:32'

In [50]:
conn = psycopg2.connect(database = "DataLab", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "",
                        port = 5432)
cur = conn.cursor()
time = series2

#Fetch the series ID for the day of the week because this will determine the time frame we check in
cur.execute("""SELECT seriesID FROM series WHERE EXTRACT(DOW FROM startdate::date) = EXTRACT(DOW FROM %s::date)
            """, (time,))
match = cur.fetchall()
conn.commit()

#Check series to see if this day of a workshop is a day with 2 workshops back to back or not as that changes our time frame we can search in
if (len(match) == 2):
    cur.execute("""SELECT startTime, endTime FROM series 
               WHERE seriesID = %s OR seriesID = %s 
            """, (match[0][0], match[1][0]))
    timeRange = cur.fetchall()
    conn.commit()
    startTime1 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[0][0] )
    endTime1 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[0][1])  
    startTime2 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[1][0])
    endTime2 = datetime.combine(datetime(year=2000, month=1, day=1),timeRange[1][1])

    #Workshop 1 ends right before Workshop 2 begins so the ranges cannot overlap
    if ((endTime1 - startTime2) < timedelta(minutes=1)):
        cur.execute("""SELECT seriesID FROM series 
               WHERE (seriesID = %s AND ((StartTime - interval '15 minutes') < time %s) AND (EndTime > time %s))
               OR (seriesID = %s AND (StartTime < time %s) AND ((EndTime + interval '15 minutes') > time %s))
            """, (match[0][0], time, time, match[1][0], time, time))
        
    #Workshop 2 ends right before Workshop 1 begins so the ranges cannot overlap
    elif((endTime2 - startTime1) < timedelta(minutes=1)):
        cur.execute("""SELECT seriesID FROM series 
               WHERE (seriesID = %s AND ((StartTime - interval '15 minutes') < time %s) AND (EndTime > time %s))
               OR (seriesID = %s AND (StartTime < time %s) AND ((EndTime + interval '15 minutes') > time %s))
            """, (match[1][0], time, time, match[0][0], time, time))
    
    #The workshops are just on the same day and do not collide
    else:
        cur.execute("""SELECT seriesID FROM series 
               WHERE ( (seriesID = %s) OR (seriesID = %s) ) AND ( ((StartTime - interval '15 minutes') < time %s) AND ((EndTime + interval '15 minutes') > time %s)) )
            """, (match[0][0], match[1][0], time, time))

else:
    cur.execute("""SELECT SeriesID FROM series WHERE ((StartTime - interval '15 minutes') < time %s) AND ((EndTime + interval '15 minutes') > time %s)
                """, (time, time))
seriesID = cur.fetchall()
conn.commit()

if (seriesID):
    cur.execute("""SELECT WorkshopID FROM workshops 
                WHERE (WorkshopDate = date %s) AND SeriesID = %s
                """, (time, seriesID[0][0]))
    workshopID = cur.fetchall()
    conn.commit()

    return workshopID
else:
    return -1

#id = getWorkshopID(checkIn.loc[0]['Timestamp'], conn, cur)

conn.close()
cur.close()

In [51]:
workshopID

[(76,)]